In [1]:
#- 패키지 임포트
import pandas as pd
import numpy as np
#import emoji # 이모지 제거용
import re
from soynlp.normalizer import * #반복되는 자음 제거
import time
from tqdm import tqdm #for문 진행률 
from time import sleep #for문 진행 시간 
from kiwipiepy import Kiwi #문장 분리2

from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)


Output()

In [2]:
''' 파일 불러오기 '''
df = pd.read_csv("Ternary_classification_data.csv", encoding='UTF-8')
# print(df.shape, type(df)) # 파일 구조 및 컬럼명 확인용
df['org_idx']= df.index # 인덱스 column 만들기: 문장 분리할 때 사용함
post_list = list(df['sentence']) #자연어 전처리를 위해 본문을 리스트 형태로

In [3]:
''' 불용어 제거 '''
# post_list 양이 많을 경우 불용어 제거 중에 오류 나면 다시 리스트로 만드는 데 오래 걸려서 카피하는 것임. 안 하고 post_list로 진행해도 됨.
post_list2 = post_list.copy()
for i in tqdm(range(len(post_list2))):
    post_list2[i] = re.sub('\n', ' ', string=str(post_list2[i])) # 줄바꿈을 띄어쓰기 하나로
    post_list2[i] = re.compile('[^|ㄱ-ㅎ|ㅏ-ㅣ|A-Z|a-z|0-9|가-힣]+').sub(' ',post_list2[i])
    post_list2[i] = repeat_normalize(post_list2[i], num_repeats=1)
	# ↑ ㅋㅋㅋ, ㅎㅎㅎ 등의 의미 없는 자음 삭제. num_repeats가 1이면 반복 글자 중 1개만 남기고 2면 2개만 남김.
    post_list2[i] = re.sub(r"^\s+|\s+$", "", post_list2[i]) # 문서 앞뒤 공백 제거
    post_list2[i] = post_list2[i].split() # 문서 내 공백(1개 이상) 기준으로 자르기
    post_list2[i] = ' '.join(post_list2[i]) # 공백 기준으로 나뉜 문서를 다시 1개의 공백을 두고 붙임.
	# ↑ 문서 내 다중 공백을 지우기 위함.]

df['cleaning'] = pd.DataFrame(post_list2) #불용어 제거된 본문을 데이터 프레임으로

100%|██████████| 616892/616892 [00:10<00:00, 59598.11it/s]


In [4]:
df

,sentence,sentiment,org_idx,cleaning
0,언니 동생으로 부르는게 맞는 일인가요..??,부정,0,언니 동생으로 부르는게 맞는 일인가요
1,그냥 내 느낌일뿐겠지?,부정,1,그냥 내 느낌일뿐겠지
2,아직너무초기라서 그런거죠?,부정,2,아직너무초기라서 그런거죠
3,유치원버스 사고 낫다던데,부정,3,유치원버스 사고 낫다던데
4,근데 원래이런거맞나요,부정,4,근데 원래이런거맞나요
...,...,...,...,...
616887,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,부정,616887,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라
616888,재미가 없으니 망하지,부정,616888,재미가 없으니 망하지
616889,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,부정,616889,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...
616890,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,부정,616890,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ


In [5]:
df2 = df[["cleaning","sentiment"]]

In [6]:
df2.rename(columns={'cleaning': 'sentence'}, inplace=True)

c:\Users\user\anaconda3\envs\NIFOS\lib\site-packages\pandas\core\frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [7]:
df2

,sentence,sentiment
0,언니 동생으로 부르는게 맞는 일인가요,부정
1,그냥 내 느낌일뿐겠지,부정
2,아직너무초기라서 그런거죠,부정
3,유치원버스 사고 낫다던데,부정
4,근데 원래이런거맞나요,부정
...,...,...
616887,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라,부정
616888,재미가 없으니 망하지,부정
616889,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,부정
616890,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,부정


In [8]:
df2.isnull().sum()

sentence     0
sentiment    0
dtype: int64

In [9]:
df2.dropna(axis=0, inplace=True)
df2.reset_index(drop=True, inplace=True)
df2.to_csv("Ternary_classification_data_spellcheck.csv", index=False, encoding='UTF-8')

c:\Users\user\anaconda3\envs\NIFOS\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
